<a href="https://colab.research.google.com/github/tak-creator/giganci/blob/main/finetunning_GPT2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, pipeline, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

In [ ]:
model_name = "flax-community/papuGaPT2"
# Tokenizer wspólny dla obu modeli
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Model przed treningiem
model_orig = GPT2LMHeadModel.from_pretrained(model_name)
model_orig.resize_token_embeddings(len(tokenizer))

# Model, który będziemy trenować
model_tuned = GPT2LMHeadModel.from_pretrained(model_name)
model_tuned.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
generator_orig = pipeline('text-generation', model=model_orig, tokenizer=tokenizer)
prompt = "gra: Stray \n opis: "
wynik = generator_orig(prompt, max_length=100, num_return_sequences=1)
print(wynik[0]['generated_text'])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


gra: Stray 
 opis: owy, który ma za zadanie zwiększyć ilość punktów do doświadczenia. W trakcie bitwy na polu bitwy znajduje się siedem przedmiotów, które mają za zadanie zwiększyć ilość punktów do doświadczenia. Są to:
- Zbroja - przy użyciu tej zbroi można podnosić i przenosić przedmioty o wartości od 1 do 10 punktów.
- Kula - po użyciu kuszy, kusza zostaje wystrzelona z przeciwnika. Dzięki temu może ona wystrzelić z łuku więcej przedmiotów, które mogą być zadane przeciwnikowi.
- Zaklęcie - broń ma na celu chronić ciało wroga. Aby go używać trzeba mieć na sobie zbroję. Po jej użyciu, broni nie można używać.
- Zręczność - broń ta jest przydatna, jednak nie można jej używać w walce. Można ją używać tylko podczas walki, nie można jej używać na arenie, w czasie walk.
Napisy: angielskie, polskie, arabskie, bułgarskie, chorwackie, czeskie, duńskie, fińskie, greckie, hebrajskie, hinduskie, węgierskie, islandzkie, norweskie, polskie, portugalskie, rumuńskie, rosyjskie, serbskie, słoweńskie, 

In [ ]:
generator_orig = pipeline("text-generation", model=model_orig, tokenizer=tokenizer)

prompt = "Gra: Hogwards Legacy\nOpis:"
wynik = generator_orig(
    prompt,
    max_new_tokens=100,
    num_return_sequences=1,
    temperature=0.9)
print("Tekst PRZED treningiem:")
print(wynik[0]["generated_text"])

Device set to use cpu


Tekst PRZED treningiem:
Gra: Hogwards Legacy
Opis: Historia życia i losów Alda Ringo o czym w końcu opowiedział w jednej z ostatnich powieści. Tak, to był najlepszy sposób, aby dowiedzieć się czegoś o tych bohaterach oraz poznać całą serię z Harrym, to tylko niektóre z nich.
Fabuła: Przeszłość zostaje odkryta zaledwie kilka lat po śmierci Alda. Jego ciało zostało wyrzucone z domu ojca i na zawsze zostało wyłowione przez matkę rodziny. Gdy Harry wraz z matką opuszczają miasto, by się ożenić, mężczyzna podejmuje


trzeba dołączyc plik TXT "gry"

In [ ]:
with open("gry.txt", "r", encoding="utf-8") as f:
    text = f.read()

texty = [blok.strip() for blok in text.split("===") if blok.strip()]

dataframe = pd.DataFrame({"text": texty})
dataset = Dataset.from_pandas(dataframe)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token #token do dopełniania -> token do sekfencji
def tokenize(batch): #batch partia danych
    tokens = tokenizer(batch["text"], padding=True, truncation=True, # padding = dopełnić sekwencje do najdłuższej lub przyciąć max i min 512!
    max_length=512)
    tokens["labels"] = tokens["input_ids"].copy() # nowy klucz dodaje labels = kopia lista tokenów wejściowych
    return tokens # zwraca słownik :)
tokenized_dataset = dataset.map(tokenize, batched=True) # nowy zbiór danych z txt na tokeny i maski do modelu

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
#Fine-tuning modelu
training_args = TrainingArguments(
    output_dir="./papuGaGPT2-finetuned",
    num_train_epochs=10, # 2 epoki
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=1,
    report_to="none"
    )
trainer = Trainer(
    model=model_tuned,
    args=training_args,
    train_dataset=tokenized_dataset,
    )
trainer.train()

Step,Training Loss
10,1.345200
20,0.311400
30,0.094800


TrainOutput(global_step=30, training_loss=0.5838111778100331, metrics={'train_runtime': 156.5594, 'train_samples_per_second': 0.383, 'train_steps_per_second': 0.192, 'total_flos': 2357752320000.0, 'train_loss': 0.5838111778100331, 'epoch': 10.0})

In [ ]:
#Generowanie opisu nowej gry po treningu

generator_tuned = pipeline("text-generation", model=model_tuned, tokenizer=tokenizer)

prompt = "Gra: Hogward Legacy\nOpis:"
wynik = generator_tuned(
    prompt,
    max_new_tokens=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.8,
    num_beams=3
    )
print(" Tekst PO treningu:")
print(wynik[0]["generated_text"])

Device set to use cpu


 Tekst PO treningu:
Gra: StarCraft 2
Opis: StarCraft to pierwsza gra RPG od George'a Lucasa, osadzona w kosmosie. Gracz wciela się w bohatera z tajemniczym pasożytem w głowie, który daje nadludzkie moce. Gra oferuje ogromną swobodę wyborów, dynamiczne walki turowe i rozbudowaną fabułę z wieloma zakończeniami.
